In [1]:
import yaml
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

with open("apisguru_openapi.yaml") as f:
    raw_apisguru_api_spec = yaml.load(f, Loader=yaml.Loader)
apisguru_api_spec = reduce_openapi_spec(raw_apisguru_api_spec)
apisguru_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://api.apis.guru/v2'}], description='Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\n', endpoints=[('GET /providers.json', 'List all the providers in the directory\n', {'description': 'List all the providers in the directory\n', 'responses': {'description': 'OK', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'data': {'type': 'array', 'items': {'type': 'string', 'minLength': 1}, 'minItems': 1}}}}}}}), ('GET /{provider}.json', 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n', {'description': 'List all APIs in the directory for a particular

In [2]:
import json

with open("mockserver_openapi.json") as f:
    raw_mockserver_api_spec = json.load(f)
mockserver_api_spec = reduce_openapi_spec(raw_mockserver_api_spec)
mockserver_api_spec

ReducedOpenAPISpec(servers=[{'url': 'http://127.0.0.1:4000'}], description='manager movie records', endpoints=[('GET /movies', None, {'parameters': [], 'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'properties': {'id': {'type': 'integer', 'title': 'Id'}, 'title': {'type': 'string', 'title': 'Title'}, 'year': {'type': 'integer', 'title': 'Year'}}, 'type': 'object', 'required': ['id', 'title', 'year'], 'title': 'Movie'}, 'title': 'Response Get Movies Movies Get'}}}}}), ('POST /movies', None, {'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'properties': {'id': {'type': 'integer', 'title': 'Id'}, 'title': {'type': 'string', 'title': 'Title'}, 'year': {'type': 'integer', 'title': 'Year'}}, 'type': 'object', 'required': ['id', 'title', 'year'], 'title': 'Movie'}}}}, 'requestBody': {'required': True, 'content': {'application/json': {'schema': {'properties': {'id': {'ty

In [3]:
servers = raw_apisguru_api_spec.get("servers", [])
base_url = servers[0]["url"] if servers else ""
base_url

'https://api.apis.guru/v2'

In [4]:
endpoints = [
    (
        f"{operation_name.upper()} {route}",
        docs.get("description", docs.get("summary", "")),
        docs,
    )
    for route, operation in raw_apisguru_api_spec["paths"].items()
    for operation_name, docs in operation.items()
]
endpoints

[('GET /providers.json',
  'List all the providers in the directory\n',
  {'operationId': 'getProviders',
   'tags': ['APIs'],
   'summary': 'List all providers',
   'description': 'List all the providers in the directory\n',
   'responses': {'200': {'description': 'OK',
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'data': {'type': 'array',
          'items': {'type': 'string', 'minLength': 1},
          'minItems': 1}}}}}}}}),
 ('GET /{provider}.json',
  'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
  {'operationId': 'getProvider',
   'tags': ['APIs'],
   'summary': 'List all APIs for a particular provider',
   'description': 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
   'parameters': [{'$ref': '#/components/parameters/provider'}],
   'responses': {'200': {'description': 'OK',
     'con

In [5]:
from langchain_core.utils.json_schema import dereference_refs

endpoints = [
    (name, description, dereference_refs(docs, full_schema=raw_apisguru_api_spec))
    for name, description, docs in endpoints
]
endpoints

[('GET /providers.json',
  'List all the providers in the directory\n',
  {'operationId': 'getProviders',
   'tags': ['APIs'],
   'summary': 'List all providers',
   'description': 'List all the providers in the directory\n',
   'responses': {'200': {'description': 'OK',
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'data': {'type': 'array',
          'items': {'type': 'string', 'minLength': 1},
          'minItems': 1}}}}}}}}),
 ('GET /{provider}.json',
  'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
  {'operationId': 'getProvider',
   'tags': ['APIs'],
   'summary': 'List all APIs for a particular provider',
   'description': 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
   'parameters': [{'name': 'provider',
     'in': 'path',
     'required': True,
     'schema': {'type': 'string',
     

In [6]:
endpoints[1]

('GET /{provider}.json',
 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
 {'operationId': 'getProvider',
  'tags': ['APIs'],
  'summary': 'List all APIs for a particular provider',
  'description': 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n',
  'parameters': [{'name': 'provider',
    'in': 'path',
    'required': True,
    'schema': {'type': 'string',
     'minLength': 1,
     'maxLength': 255,
     'example': 'apis.guru'}}],
  'responses': {'200': {'description': 'OK',
    'content': {'application/json': {'schema': {'description': 'List of API details.\nIt is a JSON object with API IDs(`<provider>[:<service>]`) as keys.\n',
       'type': 'object',
       'additionalProperties': {'description': 'Meta information about API',
        'type': 'object',
        'required': ['added', 'preferred', 'versions'],
        'properties': {'added

In [7]:
import keyword
import re


def make_valid_identifier(s: str, prefix: str = "_", fallback: str = "_") -> str:
    """
    将任意字符串转换成合法的 Python 标识符。
    - s: 原始字符串
    - prefix: 若首字符不合法，使用的前缀
    - fallback: 若结果为空时的默认名称
    """
    # 替换不合法字符为下划线
    cleaned = re.sub(r"[^0-9a-zA-Z_]", "_", s.strip())
    # 若首字符为数字，加前缀
    if cleaned and cleaned[0].isdigit():
        cleaned = prefix + cleaned
    # 若清洗后为空或全是下划线，用 fallback
    if not cleaned or cleaned.strip("_") == "":
        cleaned = fallback
    # 若和 Python 关键字冲突，加后缀避免冲突
    if keyword.iskeyword(cleaned):
        cleaned += "_"
    return cleaned

In [8]:
make_valid_identifier(endpoints[1][0])

'GET___provider__json'

In [9]:
operation_id = endpoints[1][2].get("operationId", make_valid_identifier(endpoints[1][0]))
operation_id

'getProvider'

In [10]:
summary = endpoints[1][2].get("summary", "")
summary

'List all APIs for a particular provider'

In [11]:
parameters = endpoints[1][2].get("parameters", [])
parameters

[{'name': 'provider',
  'in': 'path',
  'required': True,
  'schema': {'type': 'string',
   'minLength': 1,
   'maxLength': 255,
   'example': 'apis.guru'}}]

In [12]:
request_body = endpoints[1][2].get("requestBody", None)
request_body

In [13]:
method = "get"
path = "/{provider}.json"

In [14]:
param = parameters[0]
param

{'name': 'provider',
 'in': 'path',
 'required': True,
 'schema': {'type': 'string',
  'minLength': 1,
  'maxLength': 255,
  'example': 'apis.guru'}}

In [15]:
param_name = param["name"]
param_in = param.get("in", "query")
is_required = param.get("required", False)
description = param.get("description", "")
schema = param.get("schema", {})
if description:
    description = f"{description} (位置: {param_in})"
else:
    description = f"参数位置: {param_in}"
description

'参数位置: path'

In [16]:
from json_schema_to_pydantic import create_model

param_schema = {"type": "object", "properties": {param_name: schema}}
ParamModel = create_model(param_schema)

In [17]:
ParamModel

json_schema_to_pydantic.model_builder.DynamicModel

In [18]:
test = ParamModel()
test

DynamicModel(provider=None)

In [ ]:
from typing import Dict, Any, Type
from langchain_core.tools import BaseTool
from langchain_community.utilities.requests import RequestsWrapper
from pydantic import BaseModel

from langchain_community.tools.requests.tool import BaseRequestsTool


class BaseOpenAPITool(BaseTool):
    """OpenAPI 工具基类"""

    requests_wrapper: RequestsWrapper
    base_url: str = ""
    path: str = ""
    method: str = ""
    parameters_schema: dict | list | None = None
    request_body_schema: dict | None = None

    def _prepare_request(self, args: Dict[str, Any]) -> Dict[str, Any]:
        """准备请求参数"""
        # 分离路径参数、查询参数和请求体
        path_params = {}
        query_params = {}
        body_data = {}


        print(self.parameters_schema)
        print(self.request_body_schema)

        for parameter in self.parameters_schema:
            if parameter.get("in") == "path":
                param_name = parameter.get("name")
                path_params[param_name] = args.get(param_name)
            elif parameter.get("in") == "query":
                param_name = parameter.get("name")
                query_params[param_name] = args.get(param_name)

        if self.request_body_schema:
            for key, value in self.request_body_schema.get("properties", {}):
                body_data[key] = args.get(key)

        return {"path_params": path_params, "query_params": query_params, "body_data": body_data}

    def _prepare_url(self, path_params: Dict[str, Any]) -> str:
        """准备完整的 URL"""
        url = f"{self.base_url}{self.path}"

        # 替换路径参数
        for key, value in path_params.items():
            placeholder = f"{{{key}}}"
            if placeholder in url:
                url = url.replace(placeholder, str(value))

        return url


class RequestsGetTool(BaseRequestsTool, BaseOpenAPITool):
    """Requests GET tool"""

    name: str = "requests_get"
    """Tool name."""
    description: str
    """Tool description."""

    def _run(self, **kwargs: Any) -> str:
        # print(kwargs)
        """执行 GET 请求"""
        request_data = self._prepare_request(kwargs)
        # print(request_data)
        url = self._prepare_url(request_data["path_params"])
        # print(url)

        response = self.requests_wrapper.get(url, params=request_data["query_params"])
        return response.strip()

    async def _arun(self, **kwargs: Any) -> str:
        raise NotImplementedError("异步请求未实现")

In [20]:
tool_class = RequestsGetTool

In [21]:
description = endpoints[1][2].get("description", summary)
description

'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n'

In [22]:
tool_description = f"{description}\n\n路径: {path}\n方法: {method.upper()}"
tool_description

'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n\n\n路径: /{provider}.json\n方法: GET'

In [ ]:
if parameters:
    param_descriptions = []
    for param in parameters:
        param_name = param["name"]
        param_in = param.get("in", "query")
        param_desc = param.get("description", "")
        param_required = "必需" if param.get("required", False) else "可选"
        param_descriptions.append(f"{param_name} ({param_in}, {param_required}): {param_desc}")

    tool_description += "\n\n参数:\n" + "\n".join(param_descriptions)
tool_description

'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n\n\n路径: /{provider}.json\n方法: GET\n\n参数:\nprovider (path, 必需): '

In [24]:
requests_wrapper = RequestsWrapper()

tool = tool_class(
    name=operation_id,
    description=tool_description,
    args_schema=ParamModel,
    requests_wrapper=requests_wrapper,
    base_url=base_url,
    path=path,
    method=method,
    parameters_schema=parameters,
    request_body_schema=request_body,
    allow_dangerous_requests=True,
)

In [25]:
tool.run({"provider": "apis.guru"})

[{'name': 'provider', 'in': 'path', 'required': True, 'schema': {'type': 'string', 'minLength': 1, 'maxLength': 255, 'example': 'apis.guru'}}]
None


'{\n  "apis": {\n    "apis.guru": {\n      "added": "2015-11-26T17:52:26.000Z",\n      "info": {\n        "contact": {\n          "email": "mike.ralphson@gmail.com",\n          "name": "APIs.guru",\n          "url": "https://APIs.guru"\n        },\n        "description": "Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\\n",\n        "license": {\n          "name": "CC0 1.0",\n          "url": "https://github.com/APIs-guru/openapi-directory#licenses"\n        },\n        "title": "APIs.guru",\n        "version": "2.2.0",\n        "x-apisguru-categories": [\n          "open_data",\n          "developer_tools"\n        ],\n        "x-logo": {\n          "url": "https://api.apis.guru

In [26]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent


llm = init_chat_model("azure_openai:gpt-4o", temperature=0.0)

agent = create_react_agent(
    llm,
    [tool],
)


question = "List all APIs in the directory for apis.guru"
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

List all APIs in the directory for apis.guru
================================== Ai Message ==================================
Tool Calls:
  getProvider (call_kAt8GYPecw6OMKte37FufY5G)
 Call ID: call_kAt8GYPecw6OMKte37FufY5G
  Args:
    provider: apis.guru
[{'name': 'provider', 'in': 'path', 'required': True, 'schema': {'type': 'string', 'minLength': 1, 'maxLength': 255, 'example': 'apis.guru'}}]
None
================================= Tool Message =================================
Name: getProvider

{
  "apis": {
    "apis.guru": {
      "added": "2015-11-26T17:52:26.000Z",
      "info": {
        "contact": {
          "email": "mike.ralphson@gmail.com",
          "name": "APIs.guru",
          "url": "https://APIs.guru"
        },
        "description": "Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join o